<a href="https://colab.research.google.com/github/RO-AD/waymo-od-motion-pred/blob/main/tutorial/3_submit_test/gp-submit_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Submit Test

제출 형태도 정해져있으므로 Validation Set을 이용해서 제출해보기

## GCS(Google Cloud Stoarge)에서 Google Drive로 데이터셋 다운받기

google drive를 먼저 colab에 연동한 뒤, 해당 디렉토리에 GCS로부터 데이터셋 다운받기

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%%capture

# Validation Set만 70Gb 좀 넘어서 30분 정도 걸림. 감안 하고 실행해야 함.

#!gsutil -m cp -r \
#  "gs://waymo_open_dataset_motion_v_1_2_0/uncompressed/tf_example/validation/" \
#  /content/drive/MyDrive/waymo-od-dataset/waymo_open_dataset_motion_v_1_2_0/uncompressed/tf_example/validation/

In [2]:
drive.flush_and_unmount()

## 환경세팅

In [1]:
#%%capture
!pip3 install --upgrade pip
!pip install waymo-open-dataset-tf-2-11-0==1.5.1 # 최신 버전으로 한 번 해볼까나?

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached waymo_open_dataset_tf_2_11_0-1.5.1-py3-none-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (4.0 MB)
  Using cached tensorflow_graphics-2021.12.3-py3-none-any.whl (4.3 MB)
  Using cached immutabledict-2.2.0-py3-none-any.whl (4.0 kB)
  Using cached matplotlib-3.6.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.8 MB)
  Using cached einsum-0.3.0-py3-none-any.whl (5.1 kB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached tensorflow-2.11.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (588.3 MB)
  Using cached scikit_image-0.20.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.2 MB)
  Using cached dask-2023.3.1-py3-none-any.whl (1.2 MB)
  Using cached pyarrow-10.0.0-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (

## 데이터셋 로드

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
WOD_PATH = '/content/drive/MyDrive/waymo-od-dataset/waymo_open_dataset_motion_v_1_2_0/uncompressed'
!ls -al {WOD_PATH}

total 4
drwx------ 4 root root 4096 Apr 13 10:34 tf_example


In [21]:
VALIDSET_PATH = WOD_PATH + '/tf_example/validation'
!ls -al {VALIDSET_PATH} | wc -l

151


In [22]:
import glob

# 로컬 디렉토리에 저장된 TFRecord 파일들의 경로 리스트 생성
file_paths = glob.glob(VALIDSET_PATH + "/*.tfrecord*")
len(file_paths)

150

In [23]:
import tensorflow as tf

# TFRecordDataset 클래스를 사용하여 데이터를 읽어들임
dataset = tf.data.TFRecordDataset(file_paths, compression_type='')

In [24]:
from waymo_open_dataset import dataset_pb2

for data in dataset:
  frame = dataset_pb2.Frame()
  frame.ParseFromString(bytearray(data.numpy()))
  break



In [17]:
# 데이터셋 한 개만 읽어보기
from waymo_open_dataset.protos import scenario_pb2

for data in dataset:
  scenario = scenario_pb2.Scenario()
  scenario.ParseFromString(bytearray(data.numpy()))
  break

scenario

NameError: ignored

In [ ]:
# tutorial/3_submit_test/gp-submit_test.ipynb